In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install pyspark

     |████████████████████████████████| 204.2MB 71kB/s 
     |████████████████████████████████| 204kB 43.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=f2bdc8d1c293f24af017a9943a392f1b4967960e43b95820cb0d6bbdff87c61d
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load functionality to manipulate dataframes
from pyspark.sql import functions as fn
from pyspark.sql.functions import col, monotonically_increasing_id, split, when

# Functionality for computing features
from pyspark.ml import feature, regression, classification, Pipeline, clustering
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.feature import RFormula, Tokenizer, VectorAssembler, HashingTF, Word2Vec
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from itertools import chain
from pyspark.ml.linalg import Vectors, VectorUDT

#Evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
#flatten list spark df to spark df
def flatten_list(df):
  test_pd = df.toPandas()
  mylist1 = []
  klist = []
  for i in range(len(test_pd.kmeans_feat)):
    newlist = list(chain.from_iterable(test_pd.category[i]))
    mylist1.append(newlist)
    klist.append(i)
  transferdf_pd = pd.DataFrame({"kmeans_feat": klist, "category": mylist1})
  newcatewoo = spark.createDataFrame(transferdf_pd)
  return newcatewoo

In [ ]:
def three_highest_catpluscount(df):  
  #to pandas
  test1 = df.select("tf").toPandas()
  maxi = []
  maxv = [0,0,0]
  indexlist = []
  valuelist = []
  kmeanlist = []
  for a in range(len(test1.tf)):
    for i in range(len(test1.tf[a])):
      if (min(maxv) < test1.tf[a][i]):
        maxv[0] = test1.tf[a][i]
        maxv = sorted(maxv)
    testarr = test1.tf[a].toArray()
    for n in range(2, -1, -1):
      index = np.where(testarr == maxv[n])
      maxi.append(index[0][0])
      valuelist.append(maxv[n])
      kmeanlist.append(a)
    maxv = [0,0,0]
  kmeancate_pd = pd.DataFrame({"categoryindex": maxi, "kmean_feat": kmeanlist, "count": valuelist})
  kmeancate = spark.createDataFrame(kmeancate_pd)
  return kmeancate

In [ ]:
# add categories to index
def add_category_words(df):
  kmeancate_pd = df.toPandas()
  mylisti = []
  for t in range(len(kmeancate_pd)):
    ind = kmeancate_pd.categoryindex[t]
    mylisti.append(wordvector_catwoo[ind])
  kmeancate_pd["category"] = mylisti
  kmeancate_words = spark.createDataFrame(kmeancate_pd)
  return kmeancate_words

In [ ]:
def create_final_category(df):
  kmeancate_pd = df.toPandas()
  multiplecate = []
  categorylist = []
  kmeanlist = []
  for o in range(0,len(kmeancate_pd),3):
    multiplecate.append(kmeancate_pd["category"][o])
    if((kmeancate_pd["count"][o]*0.7 <= kmeancate_pd["count"][o+1]) and (kmeancate_pd["category"][o] != kmeancate_pd["category"][o+1])):
      multiplecate.append(kmeancate_pd["category"][o+1])
      if((kmeancate_pd["count"][o+1]*0.7 <= kmeancate_pd["count"][o+2]) and (kmeancate_pd["category"][o+1] != kmeancate_pd["category"][o+2])):
        multiplecate.append(kmeancate_pd["category"][o+2])
    categorystring = " + ".join(multiplecate)
    categorylist.append(categorystring)
    kmeanlist.append(o/3)
    multiplecate = []
  final_pd = pd.DataFrame({"category_final": categorylist, "kmean_feat": kmeanlist})
  final = spark.createDataFrame(final_pd)
  return final

In [ ]:
def blank_as_null(x):
    return when(col(x) != "", col(x)).otherwise(None)

In [ ]:
def two_highest_catpluscount(df):  
  #to pandas
  test1 = df.toPandas()
  maxi = []
  maxv = [0]
  indexlist = []
  valuelist = []
  kmeanlist = []
  for a in range(len(test1.tf)):
    for i in range(len(test1.tf[a])):
      if (min(maxv) < test1.tf[a][i]):
        maxv[0] = test1.tf[a][i]
        maxv = sorted(maxv)
    testarr = test1.tf[a].toArray()
    for n in range(1):
      index = np.where(testarr == maxv[n])
      maxi.append(index[0][0])
      valuelist.append(maxv[n])
      kmeanlist.append(test1.permalink[a])
    maxv = [0]
  kmeancate_pd = pd.DataFrame({"categoryindex": maxi, "perma": kmeanlist, "count": valuelist})
  kmeancate = spark.createDataFrame(kmeancate_pd)
  return kmeancate

In [ ]:
# add categories to index
def add_inv_words(df):
  kmeancate_pd = df.toPandas()
  mylisti = []
  for t in range(len(kmeancate_pd)):
    ind = kmeancate_pd.categoryindex[t]
    mylisti.append(inv_words[ind])
  kmeancate_pd["category"] = mylisti
  kmeancate_words = spark.createDataFrame(kmeancate_pd)
  return kmeancate_words

In [ ]:
# add categories to index
def add_round_words(df):
  kmeancate_pd = df.toPandas()
  mylisti = []
  for t in range(len(kmeancate_pd)):
    ind = kmeancate_pd.categoryindex[t]
    mylisti.append(round_words[ind])
  kmeancate_pd["category"] = mylisti
  kmeancate_words = spark.createDataFrame(kmeancate_pd)
  return kmeancate_words

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving acquisitions.csv to acquisitions.csv
Saving additions.csv to additions.csv
Saving companies.csv to companies.csv
Saving investments.csv to investments.csv
Saving rounds.csv to rounds.csv


In [ ]:
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

In [ ]:
dfacq = spark.read.format("csv").load("acquisitions.csv", delimiter = ",", header = True)
dfadd = spark.read.format("csv").load("additions.csv", delimiter = ",", header = True)
dfcom = spark.read.format("csv").load("companies.csv", delimiter = ",", header = True)
dfinv = spark.read.format("csv").load("investments.csv", delimiter = ",", header = True)
dfrou = spark.read.format("csv").load("rounds.csv", delimiter = ",", header = True)

In [ ]:
dfcom = dfcom.select(col("permalink"), col("name"), col("homepage_url"), col("category_list"), col(" market ").alias("market"), col(" funding_total_usd ").alias("funding_total_usd"), col("status"), col("country_code"), col("state_code"), col("region"), col("city"), col("funding_rounds"), col("founded_at"), col("founded_month"), col("founded_quarter"), col("founded_year"), col("first_funding_at"), col("last_funding_at"))

In [ ]:
dfacq = dfacq.select(col("company_permalink"), col("company_name"), col("company_category_list"), col("company_market"), col("company_country_code"), col("company_state_code"), col("company_region"), col("company_city"), col("acquirer_permalink"), col("acquirer_name"), col("acquirer_category_list"), col("acquirer_market"), col("acquirer_country_code"), col("acquirer_state_code"), col("acquirer_region"), col("acquirer_city"), col("acquired_at"), col("acquired_month"), col("acquired_quarter"), col("acquired_year"), col(" price_amount ").alias("price_amount"), col("price_currency_code"))

In [ ]:
dfrou = dfrou.select(col("company_permalink"), col("company_name"), col("company_category_list"), col("company_market"), col("company_country_code"), col("company_state_code"), col("company_region"), col("company_city"), col("funding_round_permalink"), col("funding_round_type"), col("funding_round_code"), col("funded_at"), col("funded_month"), col(" funded_quarter ").alias("funded_quarter"), col("funded_year"), col(" raised_amount_usd ").alias("raised_amount_usd"))

In [ ]:
display(dfcom)

DataFrame[permalink: string, name: string, homepage_url: string, category_list: string, market: string, funding_total_usd: string, status: string, country_code: string, state_code: string, region: string, city: string, funding_rounds: string, founded_at: string, founded_month: string, founded_quarter: string, founded_year: string, first_funding_at: string, last_funding_at: string]

In [ ]:
dfcom.count()

54294

In [ ]:
#show distribution of target variable
display(
  dfcom.select("status").\
  groupby(col("status")).\
  agg(fn.count("status"))
)

DataFrame[status: string, count(status): bigint]

In [ ]:
#check for null values in general and after that for each category of the target variable
dfcom.toPandas().isnull().sum()

permalink             4856
name                  4856
homepage_url          8305
category_list         8817
market                8824
funding_total_usd     4856
status                6170
country_code         10129
state_code           24133
region               10129
city                 10972
funding_rounds        4856
founded_at           15740
founded_month        15812
founded_quarter      15812
founded_year         15812
first_funding_at      4856
last_funding_at       4856
dtype: int64

In [ ]:
dfcom.filter(col("status") == "closed").toPandas().isnull().sum()

permalink               0
name                    0
homepage_url           31
category_list          78
market                 78
funding_total_usd       0
status                  0
country_code          408
state_code           1074
region                408
city                  441
funding_rounds          0
founded_at            607
founded_month         607
founded_quarter       607
founded_year          607
first_funding_at        0
last_funding_at         0
dtype: int64

In [ ]:
dfcom.filter(col("status") == "operating").toPandas().isnull().sum()

permalink                0
name                     0
homepage_url          3092
category_list         3353
market                3358
funding_total_usd        0
status                   0
country_code          4439
state_code           16696
region                4439
city                  5166
funding_rounds           0
founded_at            9165
founded_month         9232
founded_quarter       9232
founded_year          9232
first_funding_at         0
last_funding_at          0
dtype: int64

In [ ]:
dfcom.filter(col("status") == "acquired").toPandas().isnull().sum()

permalink              0
name                   0
homepage_url         254
category_list        151
market               153
funding_total_usd      0
status                 0
country_code         220
state_code           804
region               220
city                 250
funding_rounds         0
founded_at           716
founded_month        721
founded_quarter      721
founded_year         721
first_funding_at       0
last_funding_at        0
dtype: int64

In [ ]:
#get overview of different markets and their occurrence
markets = dfcom.select("market").\
  groupby(col("market")).\
  agg(fn.count("market")).\
  sort("count(market)", ascending=False)

In [ ]:
display(markets.take(10))

[Row(market=' Software ', count(market)=4620),
 Row(market=' Biotechnology ', count(market)=3688),
 Row(market=' Mobile ', count(market)=1983),
 Row(market=' E-Commerce ', count(market)=1805),
 Row(market=' Curated Web ', count(market)=1655),
 Row(market=' Enterprise Software ', count(market)=1280),
 Row(market=' Health Care ', count(market)=1207),
 Row(market=' Clean Technology ', count(market)=1200),
 Row(market=' Games ', count(market)=1182),
 Row(market=' Hardware + Software ', count(market)=1081)]

In [ ]:
#get number of unique market categories
dfcom.select("market").distinct().count()

754

In [ ]:
#find highest funding_total_usd and appropriate startups
display(
  dfcom.select("permalink", "name", "funding_total_usd").\
  sort("funding_total_usd", ascending=False)
)

DataFrame[permalink: string, name: string, funding_total_usd: string]

In [ ]:
#understand where most of the startups are located - get cities with startup occurrence
cities = dfcom.select("permalink", "name", "city").\
  groupBy("city").\
  agg(fn.count("city")).\
  sort("count(city)", ascending=False)

In [ ]:
display(cities.take(10))

[Row(city='San Francisco', count(city)=2615),
 Row(city='New York', count(city)=2334),
 Row(city='London', count(city)=1257),
 Row(city='Palo Alto', count(city)=597),
 Row(city='Austin', count(city)=583),
 Row(city='Cambridge', count(city)=554),
 Row(city='Seattle', count(city)=554),
 Row(city='Chicago', count(city)=514),
 Row(city='Los Angeles', count(city)=508),
 Row(city='Mountain View', count(city)=497)]

In [ ]:
#understand where most of the startups are located - get countries with startup occurrence
countries = dfcom.select("permalink", "name", "country_code").\
  groupBy("country_code").\
  agg(fn.count("country_code")).\
  sort("count(country_code)", ascending=False)

In [ ]:
display(countries)

DataFrame[country_code: string, count(country_code): bigint]

In [ ]:
display(
  dfcom.select("permalink", "name", "founded_year").\
  agg(fn.max("founded_year"), fn.min("founded_year"))
)

DataFrame[max(founded_year): string, min(founded_year): string]

In [ ]:
#get basic statistics for columns with numerical value - understand min, max, mean etc.
dfcom.select("funding_total_usd", "funding_rounds", "founded_year").describe().show()

+-------+-----------------+------------------+-----------------+
|summary|funding_total_usd|    funding_rounds|     founded_year|
+-------+-----------------+------------------+-----------------+
|  count|            49438|             49438|            38482|
|   mean|329.3095238095238|1.6962053481127877|2007.359128943402|
| stddev|286.1580228709764|1.2942126991245242|7.579203055906449|
|    min|             -   |                 1|             1902|
|    max|       99,99,999 |                 9|             2014|
+-------+-----------------+------------------+-----------------+




basic examination and statistics of acquisitions data

In [ ]:
display(dfacq)

DataFrame[company_permalink: string, company_name: string, company_category_list: string, company_market: string, company_country_code: string, company_state_code: string, company_region: string, company_city: string, acquirer_permalink: string, acquirer_name: string, acquirer_category_list: string, acquirer_market: string, acquirer_country_code: string, acquirer_state_code: string, acquirer_region: string, acquirer_city: string, acquired_at: string, acquired_month: string, acquired_quarter: string, acquired_year: string, price_amount: string, price_currency_code: string]

In [ ]:
dfacq.count()

55240

In [ ]:
#who are the companies who acquire the most
acquirers = dfacq.\
  select("acquirer_name").\
  groupby(col("acquirer_name")).\
  agg(fn.count("acquirer_name")).\
  sort("count(acquirer_name)", ascending=False)

In [ ]:
display(acquirers.take(10))

[Row(acquirer_name='Cisco', count(acquirer_name)=168),
 Row(acquirer_name='Google', count(acquirer_name)=165),
 Row(acquirer_name='Microsoft', count(acquirer_name)=156),
 Row(acquirer_name='IBM', count(acquirer_name)=122),
 Row(acquirer_name='Yahoo!', count(acquirer_name)=116),
 Row(acquirer_name='Oracle Corporation', count(acquirer_name)=90),
 Row(acquirer_name='Hewlett-Packard', count(acquirer_name)=88),
 Row(acquirer_name='AOL', count(acquirer_name)=64),
 Row(acquirer_name='EMC', count(acquirer_name)=63),
 Row(acquirer_name='Intel', count(acquirer_name)=59)]

In [ ]:
#statistics of price column
dfacq.select("price_amount").describe().show()

+-------+------------------+
|summary|      price_amount|
+-------+------------------+
|  count|              3716|
|   mean|141.73529411764707|
| stddev| 240.5853101164954|
|    min|              -   |
|    max|     99,60,00,000 |
+-------+------------------+



In [ ]:
#which different currencies occur
display(
  dfacq.\
  select("price_currency_code").\
  groupby("price_currency_code").\
  agg(fn.count("price_currency_code")).\
  sort("count(price_currency_code)", ascending=False)
)

DataFrame[price_currency_code: string, count(price_currency_code): bigint]

basic examination and statistics of investment data

In [ ]:
display(dfinv)

DataFrame[company_permalink: string, company_name: string, company_category_list: string, company_market: string, company_country_code: string, company_state_code: string, company_region: string, company_city: string, investor_permalink: string, investor_name: string, investor_category_list: string, investor_market: string, investor_country_code: string, investor_state_code: string, investor_region: string, investor_city: string, funding_round_permalink: string, funding_round_type: string, funding_round_code: string, funded_at: string, funded_month: string, funded_quarter: string, funded_year: string, raised_amount_usd: string]

In [ ]:
dfinv.count()

114506

In [ ]:
#check for null values
dfinv.toPandas().isnull().sum()

company_permalink              0
company_name                   0
company_category_list       3264
company_market              3266
company_country_code        7359
company_state_code         35348
company_region              7359
company_city                8705
investor_permalink            66
investor_name                 66
investor_category_list     83999
investor_market            84051
investor_country_code      27985
investor_state_code        52232
investor_region            27985
investor_city              28499
funding_round_permalink        0
funding_round_type             0
funding_round_code         59837
funded_at                      0
funded_month                   0
funded_quarter                 0
funded_year                    0
raised_amount_usd          13351
dtype: int64

In [ ]:
# get investor name, number of occurrences and total amount of invested usd to detect top investors
investors = dfinv.\
  select("investor_name", "raised_amount_usd").\
  groupby(col("investor_name")).\
  agg(fn.count("investor_name"), fn.sum("raised_amount_usd")).\
  sort("count(investor_name)", ascending=False)

In [ ]:
display(investors.take(7))

[Row(investor_name='Sequoia Capital', count(investor_name)=776, sum(raised_amount_usd)=None),
 Row(investor_name='Start-Up Chile', count(investor_name)=702, sum(raised_amount_usd)=None),
 Row(investor_name='500 Startups', count(investor_name)=694, sum(raised_amount_usd)=None),
 Row(investor_name='Intel Capital', count(investor_name)=674, sum(raised_amount_usd)=None),
 Row(investor_name='Y Combinator', count(investor_name)=625, sum(raised_amount_usd)=None),
 Row(investor_name='New Enterprise Associates', count(investor_name)=619, sum(raised_amount_usd)=None),
 Row(investor_name='Accel Partners', count(investor_name)=592, sum(raised_amount_usd)=None)]

In [ ]:
#where are most of the investors located
investorcities = dfinv.\
  select("investor_city",).\
  groupby(col("investor_city")).\
  agg(fn.count("investor_city")).\
  sort("count(investor_city)", ascending=False)

In [ ]:
display(investorcities.take(8))

[Row(investor_city='Menlo Park', count(investor_city)=9962),
 Row(investor_city='New York', count(investor_city)=7030),
 Row(investor_city='San Francisco', count(investor_city)=5934),
 Row(investor_city='Palo Alto', count(investor_city)=5224),
 Row(investor_city='London', count(investor_city)=3146),
 Row(investor_city='Boston', count(investor_city)=2637),
 Row(investor_city='Mountain View', count(investor_city)=1786),
 Row(investor_city='Cambridge', count(investor_city)=1761)]

In [ ]:
#how many distinct investors do we have
dfinv.\
select("investor_permalink").\
distinct().\
count()

22277

In [ ]:
dfinv.select("raised_amount_usd", "funded_year").describe().show()

+-------+------------------+-----------------+
|summary| raised_amount_usd|      funded_year|
+-------+------------------+-----------------+
|  count|            101155|           114506|
|   mean|149.08333333333334|2010.686173650289|
| stddev|157.12413167181924|3.065592487565398|
|    min|              -   |             1921|
|    max|           99,995 |             2014|
+-------+------------------+-----------------+



In [ ]:
#how much money per year was invested over time
investperyear = dfinv.\
select("funded_year", "raised_amount_usd").\
groupby("funded_year").\
agg(fn.sum("raised_amount_usd")).\
sort("funded_year")

In [ ]:
display(investperyear)

DataFrame[funded_year: string, sum(raised_amount_usd): double]

basic examination and statistics of rounds data

In [ ]:
display(dfrou)

DataFrame[company_permalink: string, company_name: string, company_category_list: string, company_market: string, company_country_code: string, company_state_code: string, company_region: string, company_city: string, funding_round_permalink: string, funding_round_type: string, funding_round_code: string, funded_at: string, funded_month: string, funded_quarter: string, funded_year: string, raised_amount_usd: string]

In [ ]:
dfrou.count()

83870

In [ ]:
dfrou.toPandas().isnull().sum()

company_permalink              0
company_name                   0
company_category_list       4446
company_market              4453
company_country_code        6566
company_state_code         27373
company_region              6566
company_city                7647
funding_round_permalink        0
funding_round_type             0
funding_round_code         61000
funded_at                      0
funded_month                  10
funded_quarter                10
funded_year                   10
raised_amount_usd          12831
dtype: int64

In [ ]:
#which funding round types happen more often than others - which are our main types
fundingrounds = dfrou.\
  select("funding_round_type").\
  groupBy("funding_round_type").\
  agg(fn.count("funding_round_type")).\
  sort("count(funding_round_type)", ascending=False)

In [ ]:
display(fundingrounds.take(7))

[Row(funding_round_type='venture', count(funding_round_type)=41742),
 Row(funding_round_type='seed', count(funding_round_type)=21036),
 Row(funding_round_type='debt_financing', count(funding_round_type)=5692),
 Row(funding_round_type='angel', count(funding_round_type)=4443),
 Row(funding_round_type='undisclosed', count(funding_round_type)=3871),
 Row(funding_round_type='equity_crowdfunding', count(funding_round_type)=2256),
 Row(funding_round_type='private_equity', count(funding_round_type)=1828)]

In [ ]:
dfrou.select("raised_amount_usd", "funded_year").describe().show()

+-------+------------------+-----------------+
|summary| raised_amount_usd|      funded_year|
+-------+------------------+-----------------+
|  count|             71039|            83860|
|   mean|299.36842105263156|2011.018173145719|
| stddev|  270.499579586348|2.889239747417477|
|    min|              -   |             1921|
|    max|           99,999 |             2015|
+-------+------------------+-----------------+



In [ ]:
# Replace + with | , see Hardware+Software
df_categories_pd = dfcom.select("category_list").toPandas()
k = []
for i in df_categories_pd.category_list:
    i = str(i)
    j = i.replace(' + ','|')
    k.append(j)

In [ ]:
sql = SQLContext(spark)

In [ ]:
# Create Spark DF with new Category list
mylist_pd_k = pd.DataFrame({"category_list": k})
df_cat_k = sql.createDataFrame(mylist_pd_k)

In [ ]:
#convert to Pandas
permalink_pd = dfcom.select("permalink").toPandas()

In [ ]:
df_categories_pd = mylist_pd_k

mylist = []
for i in range(len(df_categories_pd)):  
  mystring = str(df_categories_pd.category_list[i])
  myremover = mystring.split("|")
  #str_list = filter(None, myremover)
  mylist.append(myremover)

mylist_pd = pd.DataFrame({"category": mylist, "permalink": permalink_pd.permalink})
df_cat = sql.createDataFrame(mylist_pd)

In [ ]:
# counter vector + kmeans clustering + fitting and transforming
cv = feature.CountVectorizer(inputCol='category', outputCol='tf')
kmeans = clustering.KMeans(k=400, featuresCol='tf', predictionCol='kmeans_feat')
pipeline_model = Pipeline(stages=[cv, kmeans]).fit(df_cat)
df_catid_kmeans = pipeline_model.transform(df_cat)

In [ ]:
#concatenate the categories which are within one kmeans cluster
df_catid_kmeans_concat = df_catid_kmeans.\
groupby("kmeans_feat").\
agg(fn.collect_list(col("category")).alias("category")).\
sort("kmeans_feat")

In [ ]:
#execute custom functions before the following

#flatten list spark df to spark df
newcatewoo = flatten_list(df_catid_kmeans_concat)

In [ ]:
# counter vectorizer
cv = feature.CountVectorizer(inputCol='category', outputCol='tf')

In [ ]:
# cv fitting and df transformation
cv_model = cv.fit(newcatewoo)
df_catwoo_cv = cv_model.transform(newcatewoo)

In [ ]:
#get the three categories occuring most often for each cluster
kmeancate = three_highest_catpluscount(df_catwoo_cv)

In [ ]:
wordvector_catwoo = cv_model.vocabulary
wordvector_catwoo

['',
 'None',
 'Software',
 'Mobile',
 'Biotechnology',
 'E-Commerce',
 'Curated Web',
 'Social Media',
 'Enterprise Software',
 'Advertising',
 'Games',
 'Hardware',
 'Health Care',
 'Finance',
 'Education',
 'Clean Technology',
 'Analytics',
 'Health and Wellness',
 'SaaS',
 'Internet',
 'Apps',
 'Manufacturing',
 'Security',
 'Video',
 'Fashion',
 'Search',
 'Consulting',
 'Travel',
 'News',
 'Cloud Computing',
 'Web Hosting',
 'Hospitality',
 'Music',
 'Networking',
 'Technology',
 'Social Network Media',
 'Entertainment',
 'Sales and Marketing',
 'Real Estate',
 'Marketplaces',
 'Messaging',
 'Semiconductors',
 'Big Data',
 'Sports',
 'Photography',
 'iPhone',
 'Media',
 'Android',
 'Startups',
 'Medical',
 'Web Development',
 'Retail',
 'Design',
 'Automotive',
 'Content',
 'Events',
 'Location Based Services',
 'Digital Media',
 'Enterprises',
 'Facebook Applications',
 'Collaboration',
 'Publishing',
 'Reviews and Recommendations',
 'Payments',
 'Information Technology',
 'iOS'

In [ ]:
# add categories to index
kmeancate_words = add_category_words(kmeancate)

In [ ]:
#create final representative category value for each cluster
final = create_final_category(kmeancate_words)

In [ ]:
display(
  final.\
  groupby("category_final").\
  agg(fn.count("category_final").alias("count")).\
  filter(col("count") > 1).\
  sort(col("count").desc())
)

DataFrame[category_final: string, count: bigint]

In [ ]:
#get number of unique categories
final.select("category_final").distinct().count()

3

check if companies are unique

In [ ]:
#look for permalinks that occur twice 
display(dfcom.select("permalink").groupBy("permalink").agg(fn.count("permalink")).where(fn.count("permalink") == 2))

DataFrame[permalink: string, count(permalink): bigint]

In [ ]:
#investigate duplicates
display(dfcom.filter((col("permalink") == "/organization/prysm")|(col("permalink") == "/organization/treasure-valley-urology-services")))

DataFrame[permalink: string, name: string, homepage_url: string, category_list: string, market: string, funding_total_usd: string, status: string, country_code: string, state_code: string, region: string, city: string, funding_rounds: string, founded_at: string, founded_month: string, founded_quarter: string, founded_year: string, first_funding_at: string, last_funding_at: string]

remove duplicates with less information

In [ ]:
#select observation with less information of duplicate 1
dfcomsubtr1 = dfcom.filter((col("permalink") == "/organization/prysm") & ((col("permalink") == "/organization/prysm") & (col("funding_rounds") == 1)))

In [ ]:
#select observation with less information of duplicate 2
dfcomsubtr2 = dfcom.filter((col("permalink") == "/organization/treasure-valley-urology-services") & ((col("permalink") == "/organization/treasure-valley-urology-services") & (col("funding_rounds") == 1)))

In [ ]:
#get rid of duplicates (deprecated entries)
dfcom = dfcom.subtract(dfcomsubtr1).subtract(dfcomsubtr2)

In [ ]:
#double check if preparation worked out
display(dfcom.select("permalink").groupBy("permalink").agg(fn.count("permalink")).sort("count(permalink)", ascending=False))

DataFrame[permalink: string, count(permalink): bigint]

define calculations for aggregation when joining the spreadsheets

In [ ]:
#calculate the age of a company based on year of founding, the appropriate quarter and 2015, which is the date when the data set was created
age_calc = fn.when(col("quarter_new") == "Q2", 2015 - col("founded_year") - 0.25).\
  otherwise(fn.when(col("quarter_new") == "Q3", 2015 - col("founded_year") - 0.5).\
            otherwise(fn.when(col("quarter_new") == "Q4", 2015 - col("founded_year") - 0.75).\
                      otherwise(2015 - col("founded_year"))))

In [ ]:
#calculate the time to funding of a company based on date of funding and date of data set creation 
time_to_funding_calc = fn.when(col("funded_quarter_new") == "Q2", col("funded_year") + 0.25 - (2015 - col("age"))).\
  otherwise(fn.when(col("funded_quarter_new") == "Q3", col("funded_year") + 0.5 - (2015 - col("age"))).\
            otherwise(fn.when(col("funded_quarter_new") == "Q4", col("funded_year") + 0.75 - (2015 - col("age"))).\
                      otherwise(col("funded_year") - (2015 - col("age")))))

define subsets before joining

In [ ]:
#select subsets before joining to exclude information/observations which will not be used
dfcomsub = dfcom.\
  withColumn("quarter_new", col("founded_quarter").substr(6,2)).\
  withColumn("age", age_calc).\
  select("permalink", "name", "market", "funding_total_usd", "status", "country_code", "city", "funding_rounds", "founded_year", "quarter_new","age")

dfinvsub = dfinv.\
  withColumn("funded_quarter_new", col("funded_quarter").substr(6,2)).\
  select("company_permalink", "investor_permalink", "investor_name", "investor_country_code", "funding_round_type", "funded_quarter_new", "funded_year", "raised_amount_usd")

join companies and investments data

In [ ]:
#multiple rows in dfinv for each permalink
dfmaster = dfcomsub.join(dfinvsub, dfcomsub["permalink"] == dfinvsub["company_permalink"], 'leftouter')
dfmaster2 = dfmaster.\
  withColumn("time_to_funding", time_to_funding_calc)

aggregate multiple entries for each company

In [ ]:
dfmaster2_agg = dfmaster2.\
  groupby(col("permalink").alias("permalink_agg")).\
  agg(fn.count("investor_permalink").alias("count_investor"),
      fn.min("time_to_funding").alias("time_to_first_funding"),
      fn.concat_ws(", ", fn.collect_list(col("investor_country_code"))).alias("investor_country_codes"),
      fn.concat_ws(", ", fn.collect_list(col("funding_round_type"))).alias("funding_round_types"),
      fn.sum("raised_amount_usd").alias("total_raised_usd"))

In [ ]:
dfmaster2_agg.count()

49437

join aggregated values to companies subset dataframe

In [ ]:
dfmaster_final = dfcomsub.join(dfmaster2_agg, dfcomsub["permalink"] == dfmaster2_agg["permalink_agg"], 'leftouter')

In [ ]:
display(dfmaster_final)

DataFrame[permalink: string, name: string, market: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: double, permalink_agg: string, count_investor: bigint, time_to_first_funding: double, investor_country_codes: string, funding_round_types: string, total_raised_usd: double]

In [ ]:
dfmaster_final.count()

49437

In [ ]:
#examination of observations with target variable acquired
display(dfmaster.where(col("status") == "acquired"))

DataFrame[permalink: string, name: string, market: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: double, company_permalink: string, investor_permalink: string, investor_name: string, investor_country_code: string, funding_round_type: string, funded_quarter_new: string, funded_year: string, raised_amount_usd: string]

In [ ]:
#drop column which exists twice with different name
dfmaster3 = dfmaster2.drop("company_permalink")

investigate quarter of founding and funds

In [ ]:
#summarize raised amount of USD for each quarter
investperquarter = dfmaster3.\
select("quarter_new", "raised_amount_usd").\
groupby("quarter_new").\
agg(fn.sum("raised_amount_usd")).\
sort("quarter_new")

In [ ]:
investperquarternona = investperquarter.na.drop()

In [ ]:
#companies which were founded in quarter 1 tend to get more funds than others
display(investperquarternona)

DataFrame[quarter_new: string, sum(raised_amount_usd): double]

join custom category column to dataframe of companies and investments data

In [ ]:
#select permalink, category and kmeans_feat
df_join1 = df_catid_kmeans.\
  select("permalink", "category", "kmeans_feat")

In [ ]:
#join to final custom category df
df_final_permas = df_join1.join(final, df_join1["kmeans_feat"] == final["kmean_feat"], 'leftouter')

In [ ]:
display(df_final_permas)

DataFrame[permalink: string, category: array<string>, kmeans_feat: int, category_final: string, kmean_feat: double]

In [ ]:
#select subset and rename permalink column to ensure unique column names
df_final_permas_sub = df_final_permas.\
  select(col("permalink").alias("permalink_sub"),"category_final")

In [ ]:
#join custom categories to master df which is the one resulting from joining companies and investment spreadsheet (inlcuding aggregation)
df_master_final_cate = dfmaster_final.join(df_final_permas_sub, dfmaster_final["permalink"] == df_final_permas_sub["permalink_sub"], 'leftouter')

In [ ]:
display(df_master_final_cate)

DataFrame[permalink: string, name: string, market: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: double, permalink_agg: string, count_investor: bigint, time_to_first_funding: double, investor_country_codes: string, funding_round_types: string, total_raised_usd: double, permalink_sub: string, category_final: string]

get majority of investor_country_codes and funding_round_types to reduce complexity

In [ ]:
#select subset of appropriate columns
dfmastermajority = df_master_final_cate.select("permalink", "investor_country_codes", "funding_round_types")

In [ ]:
#the blank strings have to get converted to NULL for further functions
dfmajority = dfmastermajority.\
  withColumn("investor_country_codes", blank_as_null("investor_country_codes")).\
  withColumn("funding_round_types", blank_as_null("funding_round_types"))

In [ ]:
#write investor_country_codes string and funding_round_types strings that a separated by commas into a list
majority = dfmajority.\
  withColumn("investor_country_codes", split(col("investor_country_codes"), ",\s*")).\
  withColumn("funding_round_types", split(col("funding_round_types"), ",\s*"))

In [ ]:
#dropping of rows with NULL values
majoritydropinv = majority.select("permalink","investor_country_codes").na.drop()
majoritydropround = majority.select("permalink","funding_round_types").na.drop()

In [ ]:
# Counter vectorizing the investor_country_codes and funding_round_types feature in order to use the vector to calculate the majority count for each observation

cv_inv = feature.CountVectorizer(inputCol='investor_country_codes', outputCol='tf')

cv_round = feature.CountVectorizer(inputCol='funding_round_types', outputCol='tf')


cv_inv_model = cv_inv.fit(majoritydropinv)
df_cv_inv = cv_inv_model.transform(majoritydropinv)

cv_round_model = cv_round.fit(majoritydropround)
df_cv_round = cv_round_model.transform(majoritydropround)

In [ ]:
#calculate the two highest counts of the investor_country_code for each company 
invtwo = two_highest_catpluscount(df_cv_inv)

In [ ]:
#assign the words of the counter vectorizer of the investor_country_codes
inv_words = cv_inv_model.vocabulary

In [ ]:
#assign the corresponding investor_country_codes to the counts for each company
invplus_words = add_inv_words(invtwo)

In [ ]:
#calculate the two highest counts of the funding_round_types for each company 
roundtwo = two_highest_catpluscount(df_cv_round)

In [ ]:
#assign the corresponding funding_round_types to the counts for each company
round_words = cv_round_model.vocabulary

In [ ]:
#assign the corresponding funding_round_types to the counts for each company
roundpluswords = add_round_words(roundtwo)

In [ ]:
#join the "majority" investor_country_codes to the master table
masternew = df_master_final_cate.join(invplus_words.select("perma",col("category").alias("investor_country_code")), df_master_final_cate["permalink"] == invplus_words["perma"], 'leftouter')

In [ ]:
#selection in order to rename and for better joining
roundpluswords = roundpluswords.select(col("perma").alias("permaround"),col("category").alias("funding_round_type"))

In [ ]:
#join the "majority" funding_round_types to the master table
masternew = masternew.join(roundpluswords, masternew["permalink"] == roundpluswords["permaround"], 'leftouter')

In [ ]:
display(masternew)

DataFrame[permalink: string, name: string, market: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: double, permalink_agg: string, count_investor: bigint, time_to_first_funding: double, investor_country_codes: string, funding_round_types: string, total_raised_usd: double, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: string]

This dataframe "masternew" is imported in the modelling notebooks as "master.csv"

In [ ]:
#this dataframe was exported and then imported in another notebook 
#the entire project is split into two notebooks

masterdropped = masternew.drop("funding_total_usd", "permalink_agg", "investor_country_codes", "funding_round_types", "permalink_sub", "perma", "permaround")

In [ ]:
masterdropped.count()

49445

In [ ]:
display(masterdropped)

DataFrame[permalink: string, name: string, market: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: double, count_investor: bigint, time_to_first_funding: double, total_raised_usd: double, category_final: string, investor_country_code: string, funding_round_type: string]

create binary value for target variable

In [ ]:
#create new column with 1 or 0 depending on value of target variable: acquired = 1, otherwise 0
finaltarget = masterdropped.\
  withColumn("label", fn.when(col("status") == "acquired",1).otherwise(0))

In [ ]:
display(finaltarget)

DataFrame[permalink: string, name: string, market: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: double, count_investor: bigint, time_to_first_funding: double, total_raised_usd: double, category_final: string, investor_country_code: string, funding_round_type: string, label: int]

drop missing values

In [ ]:
#create subset of dataframe without missing values
finalwithoutna = finaltarget.na.drop()

Understanding of final dataset without missing values

In [ ]:
df = finalwithoutna

In [ ]:
#distribution of categories in target variable
display(
  df.select("status").\
  groupby(col("status")).\
  agg(fn.count("status"))
)

DataFrame[status: string, count(status): bigint]

In [ ]:
#class balance target label
display(
  df.select("label").\
  groupby(col("label")).\
  agg(fn.count("label"))
)

DataFrame[label: int, count(label): bigint]

In [ ]:
#which categories occur most often
categories_master = df.select("category_final").\
  groupby(col("category_final")).\
  agg(fn.count("category_final")).\
  sort("count(category_final)", ascending=False)

In [ ]:
display(categories_master.take(10))

[Row(category_final='', count(category_final)=4)]

In [ ]:
#investigate in which cities most of the startups are
cities_master = df.select("name", "city").\
  groupBy("city").\
  agg(fn.count("city")).\
  sort("count(city)", ascending=False)

In [ ]:
display(cities_master.take(10))

[Row(city='Madrid', count(city)=1),
 Row(city='Dallas', count(city)=1),
 Row(city='London', count(city)=1),
 Row(city='New York', count(city)=1)]

In [ ]:
#investigate in which countries most of the startups of our data are
countries_master = df.select("name", "country_code").\
  groupBy("country_code").\
  agg(fn.count("country_code")).\
  sort("count(country_code)", ascending=False)

In [ ]:
display(countries_master)

DataFrame[country_code: string, count(country_code): bigint]

In [ ]:
# get compnay name, number of investment and total amount of invested usd sorted by number of investors
investors_master_count = df.\
  select("name", "count_investor", "total_raised_usd", "category_final").\
  groupby(col("name")).\
  agg(fn.max("count_investor"), fn.max("total_raised_usd"), fn.first("category_final")).\
  sort("max(count_investor)", ascending=False)

In [ ]:
display(investors_master_count.take(10))

[Row(name='Teladoc', max(count_investor)=17, max(total_raised_usd)=90.0, first(category_final)=''),
 Row(name='Shot & Shop', max(count_investor)=8, max(total_raised_usd)=1608.0, first(category_final)=''),
 Row(name='Monitise', max(count_investor)=5, max(total_raised_usd)=231.0, first(category_final)=''),
 Row(name='Branching Minds', max(count_investor)=4, max(total_raised_usd)=510.0, first(category_final)='')]

In [ ]:
# get investor name, number of investment and total amount of invested usd sorted by raised usd
investors_master_usd = df.\
  select("name", "count_investor", "total_raised_usd", "category_final").\
  groupby(col("name")).\
  agg(fn.max("count_investor"), fn.max("total_raised_usd"), fn.first("category_final")).\
  sort("max(total_raised_usd)", ascending=False)

In [ ]:
display(investors_master_usd.take(10))

[Row(name='Shot & Shop', max(count_investor)=8, max(total_raised_usd)=1608.0, first(category_final)=''),
 Row(name='Branching Minds', max(count_investor)=4, max(total_raised_usd)=510.0, first(category_final)=''),
 Row(name='Monitise', max(count_investor)=5, max(total_raised_usd)=231.0, first(category_final)=''),
 Row(name='Teladoc', max(count_investor)=17, max(total_raised_usd)=90.0, first(category_final)='')]

In [ ]:
# get categories of funding rounds and the number of their occurrence
fundingrounds_master = df.\
  select("funding_round_type").\
  groupBy("funding_round_type").\
  agg(fn.count("funding_round_type")).\
  sort("count(funding_round_type)", ascending=False)

In [ ]:
display(fundingrounds_master.take(10))

[Row(funding_round_type='seed', count(funding_round_type)=2),
 Row(funding_round_type='venture', count(funding_round_type)=1),
 Row(funding_round_type='post_ipo_equity', count(funding_round_type)=1)]

In [ ]:
masternew.toPandas().to_csv('master.csv')